In [ ]:
import torch
from pathlib import Path
import numpy as np
from collections import Counter, OrderedDict
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('model/lhcb.mplstyle')

In [ ]:
from model.models import AltCNN4Layer_D35_sp as Model
from model.collectdata import collect_data, collect_truth
from model.plots import plot_truth_vs_predict
from model.efficiency import efficiency, ValueSet, pv_locations, exact_efficiency
from model.training import select_gpu
from model.core import modernize_state

#### Device configuration

In [ ]:
# device = select_gpu(2)
device = torch.device("cpu")

#### Load a dataset

In [ ]:
valfile = Path('/share/lazy/schreihf/PvFinder/Oct03_20K_val.h5')

In [ ]:
validation = collect_data(valfile,
                          batch_size=1,
                          device=device,
                          slice=slice(20000),
                          masking=True)
                          

In [ ]:
model = Model().to(device)

#### Load the model weights

Modernizing old style models if needed.

In [ ]:
state = torch.load(
    '/share/lazy/schreihf/PvFinder/models/'
    '07Jan19_AltCNN4Layer_D35_sp_300epochs_240K_lr_3em5_bs256_Alt_Loss_A_5p5/'
    '07Jan19_AltCNN4Layer_D35_sp_300epochs_240K_lr_3em5_bs256_Alt_Loss_A_5p5_199.pyt'
)

In [ ]:
del state['fc1.weight'], state['fc1.bias']

In [ ]:
state = modernize_state(model, state)

In [ ]:
model.load_state_dict(state)
model.eval()

#### Compute the model and move results to Numpy

In [ ]:
%%time
with torch.no_grad():
    outputs = model(validation.dataset.tensors[0]).cpu().numpy()
    labels = validation.dataset.tensors[1].cpu().numpy()

#### Compute the efficencies over the dataset

In [ ]:
%%time
total = ValueSet(0,0,0,0)

for label, output in zip(labels, outputs):
    total += efficiency(label, output, 5., 1e-2, .2, 3)

In [ ]:
print(total)
print()
print(total.pretty())

#### Some examples of the counting algorithm:

In [ ]:
for i in range(20):
    result = efficiency(labels[i], outputs[i], 5., 1e-2, .2, 3)
    print(format(i, '2'), result)

In [ ]:
for i in range(20):
    result = efficiency(labels[i], outputs[i], 5., 1e-2,.2, 3)
    ax = plot_truth_vs_predict(labels[i], outputs[i])
    ax.set_title(str(result))

#### Adding nTracks

In [ ]:
truth = collect_truth(valfile)

In [ ]:
eff_settings = dict(
    difference = 5., # bins
    threshold = 1e-2,
    integral_threshold = .2,
    min_width = 3    # bins
)

eff_settings_nodiff = eff_settings.copy()
del eff_settings_nodiff['difference']

In [ ]:
i = 2

print(efficiency(labels[i], outputs[i], **eff_settings))

found_values = pv_locations(outputs[i], **eff_settings_nodiff)/10-100

valid = truth.n[i] > 4
zs = truth.z[i][valid]
ns = truth.n[i][valid]

print("Found nTracks Location\tDistance")
for z,n in zip(zs, ns):
    closest = np.min(np.abs(z - found_values))
    found = closest < eff_settings['difference']/10
    print(f"{found!s:5} {n:7} {z:8.5}  {closest:8.5}")
    
target_values = pv_locations(labels[i], **eff_settings_nodiff)/10-100
target_values

In [ ]:
for i in range(100):

    eff = efficiency(labels[i], outputs[i], **eff_settings)

    found_values = pv_locations(outputs[i], **eff_settings_nodiff)/10-100

    valid = truth.n[i] > 4
    zs = truth.z[i][valid]
    ns = truth.n[i][valid]

    total_found = 0
    for z,n in zip(zs, ns):
        closest = np.min(np.abs(z - found_values))
        found = closest < eff_settings['difference']/10
        total_found += found
        
    if total_found != eff.S:
        print(i, total_found, eff.S)

In [ ]:
total_found, pvs_successful, pvs_failed = exact_efficiency(truth, outputs, **eff_settings)

In [ ]:
fr = (total_found - total.S) / ((total_found + total.S) / 2)
print(f'Fractional difference between the two methods: {fr:.2%}')

In [ ]:
# Found 49507 of 54804, added 5570 (eff 90.33%) (0.557 FP/event)
print(total)
print(f'Found {total_found} of {total.real_pvs} (eff {total_found/total.real_pvs:.2%})')

In [ ]:
arr_successful = np.array([pvs_successful[i] for i in range(100)])
arr_failed = np.array([pvs_failed[i] for i in range(100)])

In [ ]:
arr_totals = arr_successful + arr_failed
arr_totals = np.ma.array(arr_totals, mask=arr_totals==0)

arr_eff = arr_successful / arr_totals

In [ ]:
print(arr_eff[:61])
print(arr_totals[:61])

In [ ]:
fix, (ax1, ax2) = plt.subplots(2,1,figsize=(10,7), gridspec_kw={'height_ratios':(2,1)})

ax1.bar(np.arange(5,61), arr_eff[5:61], width=1, align='edge')
ax1.bar(np.arange(5), arr_eff[:5], width=1, align='edge')
ax1.set_xlabel('nTracks')
ax1.set_ylabel('Efficiency')
ax1.set_ylim(0,1.0)
ax1.set_xlim(0,61)
ax1.set_yticks(np.arange(11)/10)
ax1.set_yticks([.96,.97,.98,.99],minor=True)
ax1.set_xticks(np.arange(0,61,5))
ax1.grid(which='both')

ax2.bar(np.arange(5,61), arr_totals[5:61], width=1, align='edge')
ax2.bar(np.arange(5), arr_totals[:5], width=1, align='edge')
ax2.set_xlabel('nTracks')
ax2.set_ylabel('PVs')
ax2.set_xlim(0,61)
ax2.set_ylim(50,20000)
ax2.set_yscale('log')
ax2.set_xticks(np.arange(0,61,5))
ax2.grid(which='both')

# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='C2')
textprops = dict(color='C0', multialignment='left')
textstr = f'Found {total_found} of {total.real_pvs} (eff {total_found/total.real_pvs:.2%})'
textstr += f'\nMissed {total.FP} ({total.fp_rate:.3} per event)'
textstr += '\nAsymmetry param = 5.5'
textstr += '\nEvents in sample = 20K'

# place a text box in upper left in axes coords
ax1.text(0.95, 0.9, textstr, transform=ax1.transAxes, fontsize=14,
        verticalalignment='top', horizontalalignment='right', bbox=props, fontdict=textprops)

plt.tight_layout()
plt.savefig('effntracks.pdf')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))

ax.bar(np.arange(5,61), arr_eff[5:61], width=1, align='edge')
ax.bar(np.arange(5), arr_eff[:5], width=1, align='edge')
ax.set_xlabel('nTracks')
ax.set_ylabel('Efficiency')
ax.set_ylim(0,1.0)
ax.set_xlim(0,61)
ax.set_yticks(np.arange(11)/10)
ax.set_yticks([.96,.97,.98,.99],minor=True)
ax.set_xticks(np.arange(0,61,5))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()
plt.savefig('effntracks_bg.pdf')
plt.show()